In [1]:
import csv
import os
from io import StringIO  # Để làm việc với chuỗi như một tệp

import numpy as np
import pandas as pd

In [2]:
path = "tables/CA_WARN_REPORT"

tables = [file for file in os.listdir(path=path) if file.endswith('.csv')]
tables = sorted(tables, key=lambda x: int(x.split('_')[1].split('.')[0]))

tables = [f"{path}/{table}" for table in tables]

dfs = [pd.read_csv(file) for file in tables]

In [3]:
from check_using_api import prepare_df_for_prompt

print(prepare_df_for_prompt(dfs[0], dfs[2]))


### table_fragment_1
Headers: Notice Date,Effective,Received,Company,City,No. Of,Layoff/Closure
06/22/2015=03/25/2016=07/01/2015=Maxim Integrated Product=San Jose=150=Closure Permanent
06/30/2015=08/29/2015=07/01/2015=McGraw-Hill Education=Monterey=137=Layoff Unknown at this time
07/14/2015=09/18/2015=07/20/2015=Actavis, Inc.=Corona=45=Layoff Permanent
07/17/2015=07/13/2015=07/21/2015=American Management Services LLC=Monterey=56=Closure Permanent
07/08/2015=09/06/2015=07/08/2015=Microsoft Corporation=San Diego=129=Layoff Permanent
06/30/2015=08/31/2015=07/13/2015=First Transit=Rancho=71=Layoff Permanent
07/10/2015=07/14/2015=07/13/2015=11 Main LLC=Chico=44=Layoff Permanent


Note that:
- Number of columns is: 7 
- Data type of each column is: ['object', 'object', 'object', 'object', 'object', 'numeric', 'object']

### table_fragment_2
Headers: 03/08/2016,11/11/2016,03/10/2016,Corning Incorporated,Union City,193,Closure Permanent
03/01/2016=05/06/2016=03/11/2016=Pacific Bell Telephone 

In [4]:
def check_column_equal(df1, df2):
    if df1.shape[1] != df2.shape[1]:
        print("Hai df không có cùng số cột")
        return False
    print("Hai df có cùng số cột")
    
    if df1.columns.equals(df2.columns):
        print("Hai df có cùng tên cột")
        return True
    else:
        print("Hai df không có cùng tên cột")
        return False
    
def check_data_type_equal(df1, df2):
    pass
    
def solve_non_header_table(df, headers):
    # Convert unnamed columns to NaN in first row
    first_row_values = []
    for col in df.columns:
        if str(col).startswith('Unnamed'):
            first_row_values.append(np.nan)
        else:
            first_row_values.append(col)
            
    temp_firstrow = pd.DataFrame([first_row_values], columns=headers)
    df.columns = headers

    # insert temp_data in to the first row 
    df = pd.concat([pd.DataFrame(temp_firstrow, columns=headers), df], ignore_index=True)
    df = df.reset_index(drop=True)
    return df

def concat_df(df1, df2):
    concated_df = pd.concat([df1, df2], ignore_index=True)
    return concated_df


import csv
import pandas as pd
from io import StringIO

# Hàm trợ giúp để thực hiện logic sniffing, tránh lặp code
def _perform_csv_sniffing(csv_sample_string: str) -> bool:
    """
    Thực hiện việc "đánh hơi" trên một chuỗi CSV mẫu.
    Trả về True nếu có vẻ có header, False nếu ngược lại hoặc có lỗi.
    """
    if not csv_sample_string.strip():
        # Mẫu rỗng hoặc chỉ chứa khoảng trắng thì không có header
        return False

    sniffer = csv.Sniffer()
    try:
        # sniffer.has_header() trả về True nếu nó tin rằng có header.
        # Hàm này hoạt động dựa trên việc so sánh dòng đầu tiên với các dòng tiếp theo.
        # Nó có thể không hoàn hảo cho mọi trường hợp phức tạp hoặc mơ hồ.
        return sniffer.has_header(csv_sample_string)
    except csv.Error:
        # Nếu Sniffer lỗi (không xác định được dialect, cấu trúc quá mơ hồ),
        # coi như không thể xác nhận sự tồn tại của header chuẩn.
        # Điều này có thể xảy ra nếu mẫu quá ngắn, hoặc có ký tự/cấu trúc bất thường.
        return False

def has_header(data_source: any) -> bool:
    """
    Phát hiện xem một tệp CSV hoặc một pandas DataFrame có khả năng có header hay không.

    Đối với cả hai loại đầu vào, hàm sẽ sử dụng csv.Sniffer trên một mẫu dữ liệu.
    Đối với DataFrame, một mẫu (bao gồm cả tên cột hiện tại của DataFrame) sẽ được chuyển đổi
    thành chuỗi CSV và sau đó được "đánh hơi".

    Args:
        data_source (str or pd.DataFrame): Đường dẫn đến tệp CSV
                                           hoặc một đối tượng pandas DataFrame.

    Returns:
        bool: True nếu nguồn dữ liệu có khả năng CÓ header theo Sniffer.
              False trong các trường hợp khác (không có header, rỗng, lỗi, kiểu không hỗ trợ).
    """
    MAX_SAMPLE_ROWS_FOR_DF = 15  # Số dòng dữ liệu từ DataFrame để lấy mẫu (không tính header DF)
                                 # Tổng số dòng trong chuỗi CSV sẽ là MAX_SAMPLE_ROWS_FOR_DF + 1 (cho header DF)
    MAX_FILE_LINES_FOR_SAMPLE = 15 # Số dòng đọc từ tệp để làm mẫu

    if isinstance(data_source, pd.DataFrame):
        df = data_source
        if df.empty:
            return False # DataFrame rỗng thì không có header

        columns = df.columns.tolist()

        # Heuristic 1: Kiểm tra xem tên cột có phải là một dãy số nguyên tăng dần không
        # (ví dụ: 0, 1, 2, ...), thường là dấu hiệu pandas tự tạo header.
        is_sequential_integers = True
        if not columns: # Nếu DataFrame không có cột nào
             is_sequential_integers = False
        else:
            for i, col_name_obj in enumerate(columns):
                # Tên cột có thể là số nguyên, hoặc chuỗi chứa số. Chuyển sang chuỗi để đảm bảo.
                col_name_str = str(col_name_obj)
                try:
                    col_val = int(col_name_str)
                    if col_val != i:
                        is_sequential_integers = False
                        break
                except ValueError: # Nếu tên cột không thể chuyển thành số nguyên
                    is_sequential_integers = False
                    break
        
        if is_sequential_integers: # True nếu tất cả tên cột là dãy 0, 1, 2...
            # print("Debug: DataFrame columns are sequential integers, assuming no meaningful header.")
            return False

        # Heuristic 2: Kiểm tra nếu có tên cột nào bắt đầu bằng 'Unnamed:'
        # Đây cũng là dấu hiệu thường thấy khi pandas đọc CSV không có header
        # hoặc header có các ô trống.
        for col in columns:
            if str(col).startswith('Unnamed'):
                # print("Debug: DataFrame has 'Unnamed' column, assuming no meaningful header or problematic header.")
                return False
        
        # Lấy một phần DataFrame để chuyển thành chuỗi CSV
        # df.head(n) sẽ lấy n dòng đầu tiên của dữ liệu.
        # Khi to_csv(header=True), nó sẽ thêm 1 dòng header nữa từ df.columns.
        sample_df = df.head(MAX_SAMPLE_ROWS_FOR_DF)
        
        csv_string_buffer = StringIO()
        # Chuyển DataFrame mẫu thành chuỗi CSV, bao gồm header của DataFrame
        # và không bao gồm index của DataFrame.
        sample_df.to_csv(csv_string_buffer, header=True, index=False)
        csv_sample_from_df = csv_string_buffer.getvalue()
        
        # print(f"Debug: CSV sample from DataFrame for sniffing:\n{csv_sample_from_df[:200]}") # In ra vài trăm ký tự đầu
        return _perform_csv_sniffing(csv_sample_from_df)

    elif isinstance(data_source, str):  # Đầu vào là đường dẫn tệp
        file_path = data_source
        try:
            # Sử dụng encoding='utf-8-sig' để xử lý trường hợp file có BOM (Byte Order Mark)
            # BOM là một ký tự đặc biệt ở đầu file UTF-8 có thể gây nhiễu Sniffer nếu không được bỏ qua.
            # newline='' được khuyến nghị khi làm việc với file CSV.
            with open(file_path, 'r', newline='', encoding='utf-8-sig') as csv_file:
                sample_lines_list = []
                for i, line_content in enumerate(csv_file):
                    # Đọc tối đa MAX_FILE_LINES_FOR_SAMPLE dòng
                    if i >= MAX_FILE_LINES_FOR_SAMPLE:
                        break
                    sample_lines_list.append(line_content) # line_content đã bao gồm ký tự xuống dòng
                
                if not sample_lines_list: # File rỗng hoặc không đọc được dòng nào
                    # print(f"Debug: File '{file_path}' is empty or no lines were read for sample.")
                    return False

                csv_sample_from_file = "".join(sample_lines_list)
                # print(f"Debug: CSV sample from file '{file_path}' for sniffing:\n{csv_sample_from_file[:200]}")
                return _perform_csv_sniffing(csv_sample_from_file)
                
        except FileNotFoundError:
            # print(f"Error: File not found at '{file_path}'.")
            return False 
        except Exception as e: # Bắt các lỗi khác liên quan đến tệp (ví dụ: lỗi encoding nếu file không phải utf-8/utf-8-sig)
            # print(f"An unexpected file error occurred: {e}")
            return False
    else:
        # print(f"Error: Unsupported data_source type '{type(data_source)}'.")
        return False

In [5]:
### Giả sử rằng tất cả các table đều xuất hiện một cách tuần tự
### Giả sử table đầu tiên là nó phải ở dạng chuẩn (có headers)
from check_using_api import prepare_df_for_prompt, generate

first_table = dfs[0]
first_header = first_table.columns.tolist()
processed_tables = []

for idx, df in enumerate(dfs[1:]):
    print(f"Processing table {idx + 1} of {len(dfs) - 1}")
    if has_header(df):
        # check chung header
        if check_column_equal(first_table, df):
            # nếu bằng header thì concat
            first_table = concat_df(first_table, df)
        else:
            # nếu không thì là một table hoàn toàn mới
            print("Phát hiện table mới")
            processed_tables.append(first_table)
            first_table = df
            first_header = first_table.columns.tolist()
    else:
        # nếu không có header thì gọi api check thử (hoặc code logic sau)
        prompt = prepare_df_for_prompt(first_table, df)
        response = generate(prompt)
        if response["is_continuation"]:
            # nếu đúng thì concat
            solved_df = solve_non_header_table(df, first_header)
            first_table = concat_df(first_table, solved_df)
        else:
            # nếu sai thì có thể đây là một table mới nhưng không có header
            # cần xử lý riêng table này
            processed_tables.append(first_table)
            first_table = df  # Bắt đầu một table mới
            first_header = response["second_table_header"]
            print(f"Warning: Table {idx + 1} has no header and cannot be joined with previous table")

# Đảm bảo table cuối cùng được thêm vào processed_tables
processed_tables.append(first_table)

Processing table 1 of 4
Input tokens: 139


Output tokens: 6
Processing table 2 of 4
Input tokens: 136
Output tokens: 6
Processing table 3 of 4
Hai df không có cùng số cột
Phát hiện table mới
Processing table 4 of 4
Input tokens: 85
Output tokens: 6


In [6]:
print(len(processed_tables))

2


In [7]:
for i in range(len(processed_tables)):
    with open(f"test_table_{i}.txt", "w") as f:
        f.write(processed_tables[i].to_string(index=False))